In [253]:
import pandas
import numpy as np

from collections import defaultdict
from itertools import chain
from time import sleep

Читаем данные из преобразованного в csv файла с датасетом

In [62]:
excel_data_df = pandas.read_csv('test.csv', header=0, skip_blank_lines=True)  # /home/devuser/ROSProj/sandbox/src/tests/hacathon/
excel_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59 entries, 0 to 58
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   № поля                                59 non-null     int64  
 1   Площать, гектары                      59 non-null     float64
 2   Крутизна склона, градусы              59 non-null     object 
 3   Уровень плодородия почв               59 non-null     object 
 4   Наличие многолетних сорняков в 2020г  59 non-null     object 
 5   Статус собственности на 2020 г        59 non-null     object 
 6   Расстоние до асфальта, км             59 non-null     float64
 7   Массив полей                          59 non-null     object 
 8   2018 Культура                         59 non-null     object 
 9   2018 Гербицид                         4 non-null      object 
 10  2019 Культура                         59 non-null     object 
 11  2019 Гербицид        

In [72]:
excel_data_df.head()

,№ поля,"Площать, гектары","Крутизна склона, градусы",Уровень плодородия почв,Наличие многолетних сорняков в 2020г,Статус собственности на 2020 г,"Расстоние до асфальта, км",Массив полей,2018 Культура,2018 Гербицид,2019 Культура,2019 Гербицид,2020 Культура,2020 Гербицид,Культура 2021
0,1,125.4,1.0,Средний,-,Истекает срок,0.0,1,1,0,2,0,4,0,?
1,2,154.2,7.0,Средний,+,Собственность,0.5,1,2,0,5,0,4,0,?
2,3,205.6,0.0,Средний,-,Собственность,0.0,1,1,0,2,0,4,0,?
3,4,23.5,6.0,Высокий,-,Собственность,0.7,1,2,0,5,0,4,0,?
4,5,245.4,1.0,Средний,-,Истекает срок,0.0,1,1,0,2,0,4,0,?


In [64]:
excel_data_df['Массив полей'].unique()

array([1, 2, 3, 4])

In [74]:
excel_data_df['Уровень плодородия почв'].unique()

array(['Средний', 'Высокий', 'Низкий'], dtype=object)

Создаем константы для дальнейшей работы

In [71]:
plant_species = {
    'Свекла сах': "1",
    'Ячмень пив': "2",
    'Соя': "3",
    'Пшен оз': "4",
    'Пар': "5"
}

# Соответствие класса названию
class_to_spec = {val: key for key, val in plant_species.items()}

prices = {
    'Свекла сах': 3500.0,
    'Ячмень пив': 12000.0,
    'Соя': 30000.0,
    'Пшен оз': 16000.0,
    'Пар': 0
}

yield_coef = [
    0.8, 1.0, 1.2
]

yield_prev_spec = {
    "Пар": {"Пар": 1, "Соя": 1, "Пшен оз": 1, "Ячмень пив": 1, 'Свекла сах': 1},
    "Соя": {"Пар": 4, "Соя": 2, "Пшен оз": 3, "Ячмень пив": 4, 'Свекла сах': 2},
    "Пшен оз": {"Пар": 6, "Соя": 6, "Пшен оз": 3, "Ячмень пив": 4, 'Свекла сах': 5},
    "Ячмень пив": {"Пар": 4, "Соя": 5, "Пшен оз": 4, "Ячмень пив": 3, 'Свекла сах': 5},
    "Свекла сах": {"Пар": 40, "Соя": 50, "Пшен оз": 60, "Ячмень пив": 60, 'Свекла сах': 30},
}

areas = {
    'Центр': 1, 'Восток': 2, 'Слободка': 3, 'Дальний': 4
}
class_to_areas = {val: key for key, val in areas.items()}

Немножко преобразуем таблицу для упрощенной работы с ней

In [60]:
def gerbicid_conver(table, name):
    table[name] = table[name].replace(np.nan, 0)
    table[name] = table[name].replace("Серп", 1)

def spec_to_class(table, name):
    global plant_species
    table[name] = table[name].apply(lambda x: plant_species[x])

def area_to_class(table, name):
    global plant_species
    table[name] = table[name].apply(lambda x: areas[x])


In [63]:
gerbicid_conver(excel_data_df, "2018 Гербицид")
gerbicid_conver(excel_data_df, "2019 Гербицид")
gerbicid_conver(excel_data_df, "2020 Гербицид")

excel_data_df['Крутизна склона, градусы'] = excel_data_df['Крутизна склона, градусы'].apply(lambda x: x.replace('$', '').replace(',', '.')).astype('float')

spec_to_class(excel_data_df, '2018 Культура')
spec_to_class(excel_data_df, '2019 Культура')
spec_to_class(excel_data_df, '2020 Культура')

area_to_class(excel_data_df, 'Массив полей')





Посчитаем средние показатели посева за три года

In [197]:
def calc_means(table):
    vol_2018 = defaultdict(float)
    vol_2019 = defaultdict(float)
    vol_2020 = defaultdict(float)

    for index, row in table.iterrows():
        vol_2018[row['2018 Культура']] += row['Площать, гектары']
        vol_2019[row['2019 Культура']] += row['Площать, гектары']
        vol_2020[row['2020 Культура']] += row['Площать, гектары']

    mean_vol = defaultdict(float)
    for spec in vol_2018.keys():
        for m_d in [vol_2018, vol_2019, vol_2020]:
            mean_vol[spec] += m_d[spec]

    for spec in mean_vol.keys():
        mean_vol[spec] = mean_vol[spec] / 3.0
    return mean_vol, vol_2018, vol_2019, vol_2020

mean_vol = calc_means(excel_data_df)
mean_vol_3_years = mean_vol[0]
print(mean_vol)

(defaultdict(<class 'float'>, {'1': 813.4, '2': 1694.8999999999999, '3': 1346.2666666666667, '4': 1590.8333333333333, '5': 1395.0999999999997}), defaultdict(<class 'float'>, {'1': 885.8999999999999, '2': 1205.8000000000002, '3': 1068.3999999999999, '4': 1924.6000000000001, '5': 1755.7999999999997}), defaultdict(<class 'float'>, {'2': 1954.2999999999997, '5': 1205.8000000000002, '1': 710.0, '3': 1214.6, '4': 1755.7999999999997}), defaultdict(<class 'float'>, {'4': 1092.1, '5': 1223.6999999999998, '2': 1924.6000000000001, '3': 1755.7999999999997, '1': 844.3}))


Основные функции генетического алгоритма

In [85]:
def create_gen(chromosom_dictionary, gen_len):
    return ''.join(np.random.choice(chromosom_dictionary, gen_len))

In [178]:
def mutation(parent, chromosom_dictionary):
    child = parent[:]
    mutation_p = 1.0 / len(child)
    for i in range(len(child)):
        if np.random.random() < mutation_p:
            new_g = np.random.choice(chromosom_dictionary, 1)
            new_g = list(new_g)[0]
            list1 = list(child)
            list1[i] = new_g
            child = ''.join(list1)
    return child

def crossover(parent1, parent2):
    cross_point = np.random.choice(range(len(parent1[1:-1])), 1)[0]
    child = parent1[:cross_point]
    child += parent2[cross_point: ]
    return child

Создаем отдельный класс-модель для упрощенной работы с данными

In [246]:
class Field(object):
    def __init__(self, row, yield_prev_spec, class_to_spec):
        self.yield_prev_spec = yield_prev_spec
        self.class_to_spec = class_to_spec

        self.erosion_coef = 1.0 if row['Крутизна склона, градусы'] < 3 else 0.8

        if row['2018 Гербицид'] == 1:
            self.herbicid_coef = 1.0
        elif row['2019 Гербицид'] == 1:
            self.herbicid_coef = 0.2
        elif row['2020 Гербицид'] == 1:
            self.herbicid_coef = 0.5
        else:
            self.herbicid_coef = 1.0

        self.cluster = row['Массив полей']

        self.harvesting_coef = 1.0 if row['Расстоние до асфальта, км'] <= 1.0 else 0.8
        self.rent_coef = 0.7 if row['Статус собственности на 2020 г'] == 'Истекает срок' else 1.0
        self._weeds_coef =  0.7 if row['Наличие многолетних сорняков в 2020г'] == '+' else 1.0

        self.last_spiec = self.class_to_spec[row['2020 Культура']]
        self.field_volume = row['Площать, гектары']

        self.yield_level = {'Средний': 1.0, 'Высокий': 1.2, 'Низкий': 0.8}
        self._yield_level_coef = self.yield_level[row['Уровень плодородия почв']]

    def erosion_lose(self, g):
        """
        Эррозия
        """
        return self.erosion_coef if g == '1' else 1

    def gerbicid_lose(self, spiec):
        """
        Гербициды
        """
        return self.herbicid_coef if spiec in ['1', '3'] else 1.0

    def harvesting_lose(self, spiec):
        """
        Коэфициент дальности от дороги
        """
        return self.harvesting_coef if spiec == '3' else 1.0

    def rent_risc(self):
        """
        Потери, если истекает аренда
        """
        return self.rent_coef

    def weeds_coef(self):
        """
        Потери при сорняках
        """
        return self._weeds_coef

    def yield_level_coef(self, spiec):
        """
        Уровень плодородия почв
        """
        return self._yield_level_coef

    def get_volume_coef(self, spiec):
        """
        Урожайность культур сильно зависит от культурного растения, возделываемого в предыдущем году (таблица зависимости прилагается)
        """
        cs = self.class_to_spec[spiec]

        return self.yield_prev_spec[cs][self.last_spiec]

    def field_volume_coef(self):
        return self.field_volume


Функции оценки каждого экземпляра

In [296]:
def es(val1, val2):
    return np.exp(-np.power(val1-val2, 2)/(val2*val2))


def estimate(gen, fields, class_to_spec, prices, means):
    val = 0
    spiec_vol = defaultdict(int)
    clusters_sp = defaultdict(set)
    for i in range(len(gen)):
        g = gen[i]
        field = fields[i]

        spiec_vol[g] += field.field_volume_coef()
        clusters_sp[field.cluster].add(g)

        spiec = class_to_spec[g]
        price = prices[spiec]
        volume = field.field_volume_coef() * field.get_volume_coef(g)
        yield_c = field.yield_level_coef(g)
        weeds_coef = field.weeds_coef()
        rent_risc = field.rent_risc()
        harvesting_lose = field.harvesting_lose(g)
        gerbicid_lose = field.gerbicid_lose(g)
        erosion_lose = field.erosion_lose(g)
        field_val = yield_c * price * volume * weeds_coef * rent_risc * harvesting_lose * gerbicid_lose * erosion_lose
        val += field_val

        if i != 0:
            if g != gen[i-1] and fields[i-1].cluster == field.cluster:
                val *= 0.9

    c = 1
    for key, val in means.items():
        if spiec_vol[key] != 0:
            c = c * es(spiec_vol[key], val)
        else:
            c = c * 0.0000001

    for key, value in clusters_sp.items():
        if len(value) > 3:
            val = val * 0.5
            break

    return val * c


def calc_overall_income(gen, fields, class_to_spec, prices, means):
    val = 0
    spiec_vol = defaultdict(int)
    clusters_sp = defaultdict(set)
    for i in range(len(gen)):
        g = gen[i]
        field = fields[i]

        spiec_vol[g] += field.field_volume_coef()
        clusters_sp[field.cluster].add(g)

        spiec = class_to_spec[g]
        price = prices[spiec]
        volume = field.field_volume_coef() * field.get_volume_coef(g)
        yield_c = field.yield_level_coef(g)
        weeds_coef = field.weeds_coef()
        rent_risc = field.rent_risc()
        harvesting_lose = field.harvesting_lose(g)
        gerbicid_lose = field.gerbicid_lose(g)
        erosion_lose = field.erosion_lose(g)
        field_val = yield_c * price * volume * weeds_coef * rent_risc * harvesting_lose * gerbicid_lose * erosion_lose
        val += field_val

    return val


def estimate_with_nearest(gen, fields, class_to_spec, prices, means):
    val = 0
    spiec_vol = defaultdict(int)
    clusters_sp = defaultdict(set)
    for i in range(len(gen)):
        g = gen[i]
        field = fields[i]

        spiec_vol[g] += field.field_volume_coef()
        clusters_sp[field.cluster].add(g)

        spiec = class_to_spec[g]
        price = prices[spiec]
        volume = field.field_volume_coef() * field.get_volume_coef(g)
        yield_c = field.yield_level_coef(g)
        weeds_coef = field.weeds_coef()
        rent_risc = field.rent_risc()
        harvesting_lose = field.harvesting_lose(g)
        gerbicid_lose = field.gerbicid_lose(g)
        erosion_lose = field.erosion_lose(g)
        field_val = yield_c * price * volume * weeds_coef * rent_risc * harvesting_lose * gerbicid_lose * erosion_lose
        val += field_val

        if i != 0:
            if g != gen[i-1] and fields[i-1].cluster == field.cluster:
                val *= 0.8
    c = 1
    for key, val in means.items():
        if spiec_vol[key] != 0:
            c = c * es(spiec_vol[key], val)
        else:
            c = c * 0.0000001

    for key, value in clusters_sp.items():
        if len(value) > 3:
            val = val * 0.5
            break

    return val * c


def estimate_nearest(gen, fields):
    val = 1.0
    clusters_sp = defaultdict(set)
    for i in range(len(gen)):
        g = gen[i]
        field = fields[i]
        clusters_sp[field.cluster].add(g)

    for i in range(len(gen)):
        if i != 0:
            if g != gen[i-1] and fields[i-1].cluster == field.cluster:
                val *= 0.8

    
    for key, value in clusters_sp.items():
        if len(value) > 3:
            val = val * 0.5
            break
    return val




In [297]:
fields_count = excel_data_df.shape[0]
fields_count

59

In [298]:
fields = []
for index, row in excel_data_df.iterrows():
    f = Field(row, yield_prev_spec, class_to_spec)
    fields.append(f)

print(len(fields))

59


In [299]:
g = create_gen(list(plant_species.values()), fields_count)
print(g)
l = estimate(g, fields, class_to_spec, prices, mean_vol_3_years)
print(l)

54331214521213513331344514413314341344415512144133434414441
37.1824358422264


Генетический алгоритм

In [300]:
best = (-1, '')
best_100 = list()

In [318]:
iterations = 200
generation_len = 1000
chromosom_dictionary = list(plant_species.values())
generation = [(-1, create_gen(chromosom_dictionary, fields_count)) for _ in range(generation_len)]

for _ in range(iterations):
    new_generation = []
    for g in generation:
        val = g[0]
        gen = g[1]
        if val == -1:
            val = estimate(gen, fields, class_to_spec, prices, mean_vol_3_years)
        new_generation.append((val, gen))
    new_generation = sorted(new_generation, reverse=True)

    if best[0] < new_generation[0][0]:
        best = new_generation[0]

    
    best_100.extend(new_generation)
    best_100 = list(set(best_100))
    best_100 = sorted(best_100, reverse=True)
    best_100 = best_100[: min(100, len(best_100))]
    
    val_sum = sum([p[0] for p in new_generation])
    p = [p[0] / val_sum for p in new_generation]

    generation = []

    for _ in range(generation_len):
        if np.random.random() < 0.2:
            i = np.random.choice(range(generation_len), 1, p)[0]
            new_gen = mutation(new_generation[i][1], chromosom_dictionary)
            v = -1
        elif np.random.random() < 0.2:
            i1 = np.random.choice(range(generation_len), 1, p)[0]
            i2 = np.random.choice(range(generation_len), 1, p)[0]
            new_gen = crossover(new_generation[i1][1], new_generation[i2][1])
            v = -1
        else:         
            i = np.random.choice(range(generation_len), 1, p)[0]   
            new_gen = new_generation[i][1]
            v = new_generation[i][0]
        generation.append((val, new_gen))
    sleep(0.001)



print(best)
print(best_100[0], best_100[-1])


(693.4807624215109, '34223531142131442115445532521243345324524525243425555231412')
(693.4807624215109, '34223531142131442115445532521243345324524525243425555231412') (623.7897447912704, '54423521434223212522251555354233442144215452241352542221341')


In [319]:
def print_col(gen, class_to_spec):
    for g in gen:
        print(class_to_spec[g])

print_col(best[1], class_to_spec)
print(calc_overall_income(best[1], fields, class_to_spec, prices, mean_vol_3_years))

Соя
Пшен оз
Ячмень пив
Ячмень пив
Соя
Пар
Соя
Свекла сах
Свекла сах
Пшен оз
Ячмень пив
Свекла сах
Соя
Свекла сах
Пшен оз
Пшен оз
Ячмень пив
Свекла сах
Свекла сах
Пар
Пшен оз
Пшен оз
Пар
Пар
Соя
Ячмень пив
Пар
Ячмень пив
Свекла сах
Ячмень пив
Пшен оз
Соя
Соя
Пшен оз
Пар
Соя
Ячмень пив
Пшен оз
Пар
Ячмень пив
Пшен оз
Пар
Ячмень пив
Пар
Ячмень пив
Пшен оз
Соя
Пшен оз
Ячмень пив
Пар
Пар
Пар
Пар
Ячмень пив
Соя
Свекла сах
Пшен оз
Свекла сах
Ячмень пив
436260200.0


In [320]:
most_pricest = []
for t in best_100:
    q = t[1]
    v = calc_overall_income(q, fields, class_to_spec, prices, mean_vol_3_years)
    most_pricest.append((v, q))

most_pricest = sorted(most_pricest, reverse=True)

most_shortest = []
for t in best_100:
    q = t[1]
    v = estimate_nearest(q, fields)
    most_shortest.append((v, q))

most_shortest = sorted(most_shortest, reverse=True)

most_optimal = []
for t in best_100:
    q = t[1]
    v1 = estimate_nearest(q, fields)
    v2 = calc_overall_income(q, fields, class_to_spec, prices, mean_vol_3_years)
    most_optimal.append((v1 * v2, q))

most_optimal = sorted(most_optimal, reverse=True)

print(best)
print(most_pricest[0], most_shortest[0], most_optimal[0])


(693.4807624215109, '34223531142131442115445532521243345324524525243425555231412')
(483354992.0, '33412133443442545351225425454553141435522153442543343213251') (0.08388608000000004, '14235122255124143415212222134355531331542353232154422524444') (31927866.815938577, '14235122255124143415212222134355531331542353232154422524444')


Сохранение в Exel, листы Результаты и Прибыль

In [322]:
def convert_to_names(gen, class_to_spec):
    converted = []
    for g in gen:
        converted.append(class_to_spec[g])
    return converted 

most_pricest_converted = convert_to_names(most_pricest[0][1], class_to_spec)
most_shortest_converted = convert_to_names(most_shortest[0][1], class_to_spec)
most_optimal_converted = convert_to_names(best[1], class_to_spec)

df1 = pandas.DataFrame([p for p in zip(range(1, len(most_optimal_converted) + 1), most_optimal_converted, most_pricest_converted, most_shortest_converted)],
                       columns=['№ поля', 'Самый оптимальный', 'Который принесет самую большую прибыль', 'Оптимальный по кластеризации'])

most_pricest_profit = calc_overall_income(most_pricest[0][1], fields, class_to_spec, prices, mean_vol_3_years)
most_shortest_profit = calc_overall_income(most_shortest[0][1],  fields, class_to_spec, prices, mean_vol_3_years)
most_optimal_profit = calc_overall_income(best[1],  fields, class_to_spec, prices, mean_vol_3_years)

df2 = pandas.DataFrame([most_optimal_profit, most_pricest_profit, most_shortest_profit],
                       columns=['Потенциальная прибыль'],
                       index=['Самый оптимальный', 'Который принесет самую большую прибыль', 'Оптимальный по кластеризации'])

with pandas.ExcelWriter('output.xlsx') as writer:  
    df1.to_excel(writer, sheet_name='Результаты')
    df2.to_excel(writer, sheet_name='Прибыль')